# Tree-based Analyses

Decision trees work kind of like a game of 20 Questions. They consider each variable in a dataset and finding its "split point." Variables with more predictive power will be used earlier in the splitting process. For example, imagine a set of data showing the freshness of various vegetables, and we want to predict whether they have spoiled or not. If one of the features is the farm of origin, and 80% of spoiled vegetables come from one particular farm, then the farm-of-origin feature will have high predictive value and be used early on in the tree-building process. From there, additional features would be considered to determine their value in predicting freshness. 

## Decision Trees

When using a simple tree, it is easy to produce a visualization (with a tree shape, of course) that shows splits and features in decreasing order of importance moving down the "tree." However, in this particular instance, I'm using AdaBoost, which is version of boosted trees that offers more powerful predictions. The output in this case is a bit different.

In [1]:
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score
import pandas as pd

#import mapped version of data set here
readmit = pd.read_csv('diabetes_readmission_onehot.csv') 

In [2]:
# capture independent variables in list
features = list(readmit)
features = [e for e in features if e not in ('Unnamed: 0', 'readmit30')]

In [3]:
# split the data into a training and test set
X = readmit[features].values
y = readmit.readmit30.values
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, y, test_size = .2, 
                                                random_state = 31, stratify = y)

In [4]:
# build model on training data
dt = DecisionTreeClassifier() 
clf = AdaBoostClassifier(base_estimator = dt, n_estimators = 50, learning_rate = 1, 
                         random_state = 7)
model = clf.fit(Xtrain, Ytrain)

# check performance (model accuracy) on test data
print(model.score(Xtest, Ytest))

0.847856982572


## Confusion Matrix and Recall Score

Another way to evaluate model performance is a confustion matrix, which puts our model's predictions into four categories:

- In the top-left quadrant is the number of observations classified as not readmitted within 30 days that were in fact not readmitted within 30 days. This is the true negative count. 
- In the top-right quadrant is the number of observations classified as readmitted within 30 days that were in fact not readmitted within 30 days. This is the false positive count. 
- In the lower left quadrant is the number of observations classified as not readmitted within 30 days that were in fact readmitted within 30 days. This is the false negative count. 
- In the lower right quadrant is the number of observations classified as readmitted within 30 days that were in fact readmitted within 30 days. This is the true positive count. 

In [8]:
# set actual and predicted vectors; generate confusion matrix for test data
actual = pd.Series(Ytest, name = 'Actual')
predicted = pd.Series(clf.predict(Xtest), name = 'Predicted')
train_ct = pd.crosstab(actual, predicted, margins = True)
print(train_ct)

Predicted      0     1    All
Actual                       
0          11062  1092  12154
1            942   273   1215
All        12004  1365  13369


In [9]:
 # as percentages
TN = train_ct.iloc[0,0] / train_ct.iloc[0,2]
TP = train_ct.iloc[1,1] / train_ct.iloc[1,2]
print('Accuracy for not readmitted: {}'.format('%0.3f' % TN))
print('Accuracy for readmitted (Recall): {}'.format('%0.3f' % TP))

Accuracy for not readmitted: 0.910
Accuracy for readmitted (Recall): 0.225


Finally, we can consider the model's recall score. Recall is a (percentage) measure of how many positive cases were identified correctly. In the context of this analysis, if 100 patients were readmitted within thirty days and the model classified 81 of them as such, then the model's recall would be .81 (or 81%). 

## Improvement Through Undersampling

The AdaBoost algorithm had subpar performance metrics, as evidenced by recall scores (although their accuracy scores were pretty good). The cause may be the significant imblance in positive and negative outcomes. As we saw earlier, there are about 11 patients who were not readmitted within 30 days for every patient who was. In order to see if our model's performance is due to this imbalance, we can try the undersampling process on observations that did not show readmission within 30 days. Here, we'll use random undersampling to randomly choose fewer observations with the negative outcome in order to create a balance with positive outcomes. 

In [10]:
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler 

X = readmit[features].values #majority class to be undersampled
Y = readmit.readmit30.values 

rus = RandomUnderSampler(random_state = 31)
X_res, Y_res = rus.fit_sample(X, Y)
Counter(Y_res) 

Counter({0: 6074, 1: 6074})

In [11]:
# train, test, split
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X_res, Y_res, test_size = .2, 
                                                random_state = 31, stratify = Y_res)

In [12]:
# run AdaBoost again w/ undersampled dataset
dt_rus = DecisionTreeClassifier() 
clf_rus = AdaBoostClassifier(base_estimator = dt_rus, n_estimators = 50, learning_rate = 1, 
                             random_state = 7)
model = clf_rus.fit(Xtrain, Ytrain)

In [13]:
# confusion matrix for random forest with random undersampling
actual = pd.Series(Ytest, name = 'Actual')
predicted_rus = pd.Series(clf_rus.predict(Xtest), name = 'Predicted')
ct_rus = pd.crosstab(actual, predicted_rus, margins = True)
print(ct_rus)

Predicted     0     1   All
Actual                     
0           805   410  1215
1           430   785  1215
All        1235  1195  2430


In [14]:
# confusion matrix with percentages
TN_rus = ct_rus.iloc[0,0] / ct_rus.iloc[0,2]
TP_rus = ct_rus.iloc[1,1] / ct_rus.iloc[1,2]
print('AdaBoost accuracy for not readmitted: {}'.format('%0.3f' % TN_rus))
print('AdaBoost accuracy for readmitted (Recall): {}'.format('%0.3f' % TP_rus))

AdaBoost accuracy for not readmitted: 0.663
AdaBoost accuracy for readmitted (Recall): 0.646


The undersampling approach improves the model significantly with respect to positive cases. It's noticeably better than random chance would be, too. Before finishing, we can also compare these results with another technique for handling imbalanced data: oversampling.

## Oversampling (SMOTE)

In addition to undersampling the majority class (not readmitted within 30 days), we can also try oversampling the minority class (readmitted within 30 days). Here, we'll use a common oversampling method called SMOTE (Synthetic Minority Oversampling Technique). 

In [15]:
from imblearn.over_sampling import SMOTE 

X = readmit[features].values 
Y = readmit.readmit30.values #minority class to be oversampled

sm = SMOTE(random_state = 31)
X_resamp, Y_resamp = sm.fit_sample(X, Y)
Counter(Y_resamp) 

Counter({0: 60770, 1: 60770})

In [16]:
# train, test, split
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X_resamp, Y_resamp, test_size = .2, 
                                                random_state = 31, stratify = Y_resamp)

In [1]:
# run AdaBoost again w/ oversampled dataset
clf_sm = AdaBoostClassifier(base_estimator = dt, n_estimators = 50, learning_rate = 1, 
                         random_state = 7)
model_sm = clf_sm.fit(Xtrain, Ytrain)
print(model_sm.score(Xtest, Ytest))

NameError: name 'AdaBoostClassifier' is not defined

In [18]:
# confusion matrix with SMOTE oversampling
actual = pd.Series(Ytest, name = 'Actual')
predicted_sm = pd.Series(clf_sm.predict(Xtest), name = 'Predicted')
ct_sm = pd.crosstab(actual, predicted_sm, margins = True)
print(ct_sm)

Predicted      0      1    All
Actual                        
0          12062     92  12154
1           1121  11033  12154
All        13183  11125  24308


In [19]:
# confusion matrix with percentages
TN_sm = ct_sm.iloc[0,0] / ct_sm.iloc[0,2]
TP_sm = ct_sm.iloc[1,1] / ct_sm.iloc[1,2]
print('Accuracy for not readmitted: {}'.format('%0.3f' % TN_sm))
print('Accuracy for readmitted (Recall): {}'.format('%0.3f' % TP_sm))

Accuracy for not readmitted: 0.992
Accuracy for readmitted (Recall): 0.908


Both accuracy and recall have been greatly improved from the model using undersampling. In our case, AdaBoost works best with oversampling the minority class (readmitted within 30 days) rather than undersampling the majority class (not readmitted within 30 days). This may stem from the fact that oversampling generates more overall data, which generally improves the machine-learning process since the model has more data to "learn" from. 

## Random Forest

Individual decision trees are weak learners, meaning that their accuracy is limited (often not much higher than 50%). In order to improve accuracy, we can take an ensemble approach, random forest, that combines multiple trees. The idea is that many weak learners combine their "knowledge" to create a strong learner, which is much more accurate. 

In [20]:
# split the data into a training and test set
X = readmit[features].values
y = readmit.readmit30.values
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, y, test_size = .2, 
                                                random_state = 31, stratify = y)

In [21]:
# build and fit model with random forest
clf_rf = RandomForestClassifier(random_state = 7, class_weight = {0: .1, 1: .9})
model_rf = clf_rf.fit(Xtrain, Ytrain)

In [22]:
# model accuracy on test data
print(model_rf.score(Xtest, Ytest))

0.906350512379


In [23]:
# confusion matrix
actual = pd.Series(Ytest, name = 'Actual')
predicted_rf = pd.Series(clf_rf.predict(Xtest), name = 'Predicted')
rf_ct = pd.crosstab(actual, predicted_rf, margins = True)
print(rf_ct)

Predicted      0   1    All
Actual                     
0          12098  56  12154
1           1196  19   1215
All        13294  75  13369


In [24]:
# confusion matrix with percentages
TN_rf = rf_ct.iloc[0,0] / rf_ct.iloc[0,2]
TP_rf = rf_ct.iloc[1,1] / rf_ct.iloc[1,2]
print('Accuracy for not readmitted: {}'.format('%0.3f' % TN_rf))
print('Accuracy for readmitted (Recall): {}'.format('%0.3f' % TP_rf))

Accuracy for not readmitted: 0.995
Accuracy for readmitted (Recall): 0.016


## Improvement Through Undersampling: Random Forest

The initial random forest did not show great results. Let's see if the undersampling method helps. 

In [25]:
X = readmit[features].values #majority class to be undersampled
Y = readmit.readmit30.values 

rus = RandomUnderSampler(random_state = 34)
X_res, Y_res = rus.fit_sample(X, Y)
Counter(Y_res) 

Counter({0: 6074, 1: 6074})

In [26]:
# train, test, split
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X_res, Y_res, test_size = .2, 
                                                random_state = 34, stratify = Y_res)

In [27]:
# run random forest again w/ undersampled dataset
rf_rus = RandomForestClassifier(random_state = 34)
rf_model_rus = rf_rus.fit(Xtrain, Ytrain)
print(rf_model_rus.score(Xtest, Ytest))

0.707407407407


In [28]:
# confusion matrix for random forest with random undersampling
actual = pd.Series(Ytest, name = 'Actual')
predicted_rf_rus = pd.Series(rf_rus.predict(Xtest), name = 'Predicted')
ct_rf_rus = pd.crosstab(actual, predicted_rf_rus, margins = True)
print(ct_rf_rus)

Predicted     0     1   All
Actual                     
0           912   303  1215
1           408   807  1215
All        1320  1110  2430


In [29]:
# confusion matrix with percentages
TN_rf_rus = ct_rf_rus.iloc[0,0] / ct_rf_rus.iloc[0,2]
TP_rf_rus = ct_rf_rus.iloc[1,1] / ct_rf_rus.iloc[1,2]
print('Accuracy for not readmitted: {}'.format('%0.3f' % TN_rf_rus))
print('Accuracy for readmitted (Recall): {}'.format('%0.3f' % TP_rf_rus))

Accuracy for not readmitted: 0.751
Accuracy for readmitted (Recall): 0.664


The accuracy for non-readmitted patients has dropped, which may be due to the fact that we had less data from that class this time. But the performance in that class is still well above a random guess. More importantly, the accuracy with respect to positive cases (readmitted within 30 days) is vastly improved. 

## Oversampling: Random Forest

Here, we'll try the oversampling method with the Random Forest classifier, too.

In [30]:
X = readmit[features].values 
Y = readmit.readmit30.values #minority class to be oversampled

sm = SMOTE(random_state = 137)
X_resamp, Y_resamp = sm.fit_sample(X, Y)
Counter(Y_resamp) 

Counter({0: 60770, 1: 60770})

In [31]:
# train, test, split
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X_resamp, Y_resamp, test_size = .2, 
                                                random_state = 137, stratify = Y_resamp)

In [32]:
# run random forest again w/ oversampled dataset
clf_rf_sm = RandomForestClassifier(random_state = 137)
model_rf_sm = clf_rf_sm.fit(Xtrain, Ytrain)
print(model_rf_sm.score(Xtest, Ytest))

0.950674675004


In [33]:
# confusion matrix for random forest with SMOTE oversampling
actual = pd.Series(Ytest, name = 'Actual')
predicted_rf_sm = pd.Series(clf_rf_sm.predict(Xtest), name = 'Predicted')
ct_rf_sm = pd.crosstab(actual, predicted_rf_sm, margins = True)
print(ct_rf_sm)

Predicted      0      1    All
Actual                        
0          12063     91  12154
1           1108  11046  12154
All        13171  11137  24308


In [34]:
# confusion matrix with percentages
TN_rf_sm = ct_rf_sm.iloc[0,0] / ct_rf_sm.iloc[0,2]
TP_rf_sm = ct_rf_sm.iloc[1,1] / ct_rf_sm.iloc[1,2]
print('Accuracy for not readmitted: {}'.format('%0.3f' % TN_rf_sm))
print('Accuracy for readmitted (Recall): {}'.format('%0.3f' % TP_rf_sm))

Accuracy for not readmitted: 0.993
Accuracy for readmitted (Recall): 0.909


The accuracy for patients not readmitted is nearly perfect in the SMOTE model, and its accuracy for positive cases is excellent as well. This model seems like our best one. 

In [ ]:
# feature analysis

# summary of feature importance
importance = model.feature_importances_
importance_df = pd.DataFrame({'feature': features, 'importance': importance})
imp = importance_df.sort_values('importance', ascending = False)
imp.head()